In [1]:
%matplotlib inline

import sys

import datacube
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json


import matplotlib as mpl
import matplotlib.colors
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap
from colorspacious import cspace_converter
from collections import OrderedDict
from branca.colormap import linear

cmaps = OrderedDict()

In [2]:
shp_file_location = 'ABS_shapefiles/SA2_2016_AUST/SA2_2016_AUST.shp'

csv_file_location = 'ABS_csv_files/Northern Australia SA2s in non-remote areas.csv' 

df = pd.read_csv(csv_file_location)
df = df.set_index("SA2 code")
df = df.sort_index()

gdf = gpd.read_file(shp_file_location)
gdf = gdf.rename(columns={'SA2_MAIN16':'SA2'})
gdf = gdf.set_index('SA2')
gdf = gdf.sort_index()
gdf = gdf.drop(columns=['SA2_5DIG16', 'SA2_NAME16', 'SA3_CODE16', 'SA3_NAME16', 'SA4_CODE16', 'SA4_NAME16',
                        'GCC_CODE16', 'GCC_NAME16', 'STE_CODE16', 'STE_NAME16', 'AREASQKM16'])

SA2_index = df.index

SA2_list = SA2_index.tolist()
SA2_string = str(SA2_list)
SA2 = SA2_string.replace("[","").replace("]","").replace(" ","")
SA2 = SA2.split(",")

gdf = gdf[gdf.index.isin(SA2)]
new_gdf = gdf.merge(df, on=gdf.index)
new_gdf = new_gdf.rename(columns={'key_0':'SA2'})
new_gdf = new_gdf.set_index('SA2')
new_gdf.head()

,geometry,Name
SA2,,
306011138,"POLYGON ((145.70905 -16.89652, 145.70857 -16.8...",Brinsmead
306011139,"MULTIPOLYGON (((145.69480 -16.73170, 145.69450...",Clifton Beach - Kewarra Beach
306011140,"POLYGON ((145.72333 -16.87656, 145.72335 -16.8...",Freshwater - Stratford
306011141,"POLYGON ((145.66333 -16.88106, 145.66303 -16.8...",Redlynch
306011142,"POLYGON ((145.68226 -16.79579, 145.68076 -16.7...",Trinity Beach - Smithfield


In [3]:
#Converting the data to json
data = json.loads(new_gdf.to_json())

map = ipyl.Map(center=[-19.3, 134], zoom=5)

label = ipyw.Label(layout=ipyw.Layout(width='100%'))

for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.5
    }
layer = ipyl.GeoJSON(data=data, hover_style={'fillColor': 'red'})

def click_handler(event=None, feature=None, id=None, properties=None):
    label.value = str(properties['Name'])
    
    
layer.on_hover(click_handler)
map.add_layer(layer)


ipyw.VBox([map, label])